In [1]:
%load_ext autoreload
%load_ext memory_profiler 
%load_ext dotenv
%autoreload 2
%dotenv

In [11]:
from geovision.data.imagenet import ImagenetETL
ImagenetETL.load('index', 'archive', 'imagenet_1k')

#import h5py
#import tarfile
#import pandas as pd
#import imageio.v3 as iio
#import matplotlib.pyplot as plt

#from pathlib import Path

#archive_path = Path.home() / "datasets" / "imagenet" / "staging" / "imagenette2.tgz"
#with tarfile.open(archive_path) as tf:
    #images = tf.getnames()

#df = pd.DataFrame({"image_path": [n for n in images if n.endswith(".JPEG")]})

#with tarfile.open(archive_path) as tf:
    #image = tf.extractfile(df["image_path"].iloc[0])
    #image = iio.imread(image)
    #plt.imshow(image)

,image_path,label_synset,label_idx,label_str
0,ILSVRC/Data/CLS-LOC/train/n01440764/n01440764_...,n01440764,0,tench
1,ILSVRC/Data/CLS-LOC/train/n01440764/n01440764_...,n01440764,0,tench
2,ILSVRC/Data/CLS-LOC/train/n01440764/n01440764_...,n01440764,0,tench
3,ILSVRC/Data/CLS-LOC/train/n01440764/n01440764_...,n01440764,0,tench
4,ILSVRC/Data/CLS-LOC/train/n01440764/n01440764_...,n01440764,0,tench
...,...,...,...,...
1331162,ILSVRC/Data/CLS-LOC/train/n15075141/n15075141_...,n15075141,999,toilet paper
1331163,ILSVRC/Data/CLS-LOC/train/n15075141/n15075141_...,n15075141,999,toilet paper
1331164,ILSVRC/Data/CLS-LOC/train/n15075141/n15075141_...,n15075141,999,toilet paper
1331165,ILSVRC/Data/CLS-LOC/train/n15075141/n15075141_...,n15075141,999,toilet paper


In [ ]:
import logging
from lightning import Trainer
from geovision.experiment.config import ExperimentConfig
from geovision.data.interfaces import ImageDatasetDataModule
from geovision.models.interfaces import ClassificationModule

from geovision.io.local import FileSystemIO as fs
from geovision.experiment.loggers import (
    get_csv_logger, 
    get_ckpt_logger,
    get_classification_logger
)

In [ ]:
config = ExperimentConfig.from_yaml("config.yaml")
datamodule = ImageDatasetDataModule(config.dataset_constructor, config.dataset_config, config.dataloader_config)

logging.basicConfig(
    filename = f"{fs.get_new_dir(config.experiments_dir, "logs")}/logfile.log",
    filemode = "a",
    format = "%(asctime)s : %(name)s : %(levelname)s : %(message)s",
    level = logging.INFO
)

loggers: list = list()
loggers.append(csv_logger := get_csv_logger(config))

callbacks: list = list()
callbacks.append(ckpt_logger := get_ckpt_logger(config))
callbacks.append(metrics_logger := get_classification_logger(config))

In [ ]:
config = ExperimentConfig.from_yaml("config.yaml")
model = ClassificationModule(
    model_config=config.model_config,
    criterion_constructor=config.criterion_constructor, 
    criterion_params=config.criterion_params, 
    optimizer_constructor=config.optimizer_constructor,
    optimizer_params=config.optimizer_params,
    lr_scheduler_constructor=config.scheduler_constructor,
    lr_scheduler_params=config.scheduler_params,
    warmup_scheduler_constructor=config.warmup_scheduler_constructor,
    warmup_scheduler_params=config.warmup_scheduler_params,
    warmup_steps=config.warmup_steps,
    scheduler_config_params=config.scheduler_config_params
)
trainer = Trainer(logger = loggers, callbacks = callbacks, **config.trainer_params)
trainer.fit(
    model = model, 
    datamodule = datamodule, 
    ckpt_path = config.ckpt_path 
)